In [6]:
import gzip

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Word2Vec, LdaMulticore
from gensim.corpora import Dictionary

import numpy as np
import pyLDAvis

from pyLDAvis import gensim

ModuleNotFoundError: No module named 'utils'

In [4]:
!dir data

 Wolumin w stacji C to OSDisk
 Numer seryjny woluminu: 949D-8C06

 Katalog: C:\Users\arkedzie\Documents\__prv\_#5DWCHALLENGE\Challenge_4\Ch4d3\data

2019-09-03  10:30    <DIR>          .
2019-09-03  10:30    <DIR>          ..
2019-09-03  10:30       240˙342˙787 reviews_data.txt
               1 plik(˘w)        240˙342˙787 bajt˘w
               2 katalog(˘w)  10˙022˙752˙256 bajt˘w wolnych


In [10]:
f = open("data/reviews_data.txt", "r")
for i,line in enumerate(f):
    print(i,line)
    break

0 Oct 12 2009 	Nice trendy hotel location not too bad.	I stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Beij

In [24]:
def preprocessing(sentence):
    return [word for word in simple_preprocess(sentence) if word not in STOPWORDS]
    
def read_sentences(filename):
    f = open(filename, "rb")
    for line in f:
        yield preprocessing(line)


In [25]:
sentences  = list(read_sentences("data/reviews_data.txt"))

In [26]:
len(sentences)

255404

# Model

In [27]:
%time model = Word2Vec(sentences, size=100, window=5, min_count=2)

Wall time: 1min 57s


In [28]:
#model = Word2Vec.load('models/hotel_model_size300_window3_mincount5.model')

# Reprezentacja wektorowa

In [30]:
model.wv.most_similar("good")

[('decent', 0.8266524076461792),
 ('great', 0.8133701086044312),
 ('excellent', 0.8007971048355103),
 ('ok', 0.6560794115066528),
 ('fair', 0.6283526420593262),
 ('average', 0.6129156947135925),
 ('terrific', 0.6117709279060364),
 ('reasonable', 0.6033675074577332),
 ('fantastic', 0.5993119478225708),
 ('nice', 0.5982304811477661)]

In [31]:
model.wv.most_similar("bad")

[('terrible', 0.7232615947723389),
 ('horrible', 0.6716300249099731),
 ('awful', 0.6686336994171143),
 ('negative', 0.6304006576538086),
 ('okay', 0.6038426160812378),
 ('ok', 0.5955401659011841),
 ('poor', 0.5955379605293274),
 ('alright', 0.5558809041976929),
 ('worse', 0.5545010566711426),
 ('positive', 0.5536491274833679)]

In [32]:
model.wv.most_similar(positive=["bad","horrible"], negative=["good"] )

[('awful', 0.7820453643798828),
 ('terrible', 0.7589058876037598),
 ('horrific', 0.6873496770858765),
 ('nasty', 0.6840726137161255),
 ('disgusting', 0.6819630861282349),
 ('horrid', 0.6741217374801636),
 ('dreadful', 0.6656562089920044),
 ('shocking', 0.662981390953064),
 ('miserable', 0.6563583612442017),
 ('horrendous', 0.6484431624412537)]

# Topic modeling - LDA

In [35]:
sentences_light = np.random.permutation(sentences)

In [36]:
sentences_light = sentences_light[:1000]

In [37]:
%time dictionary = Dictionary(sentences_light)

Wall time: 122 ms


In [38]:
%time bow_corpus = [dictionary.doc2bow(sent) for sent in sentences]

Wall time: 19.3 s


In [42]:
#Id tokentu - tu car
dictionary.token2id['car']
#ilosc wystapien id w zdaniu [(id,ilosc)]
dictionary.doc2bow(['car'])

[(643, 1)]

In [ ]:
%time lda_model = LdaMulticore(bow_corpus, id2word=dictionary, num_topics=100, passes=20, workers=4)

In [ ]:
for idx,topic in lda_model.print_topics(-1):
    print('Topic {} \nWords: {}'.format(idx,topic))

# Wizualizacja tematów

In [ ]:
lda_vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_vis)